Load API Keys

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from HLB Fixed Deposite Pages

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit.html?icp=hlb-en-all-footer-txt-fd",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/e-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/senior-savers-flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/foreign-fixed-deposit-account.html",
  "https://www.hlb.com.my/en/personal-banking/help-support/fees-and-charges/deposits.html",
])
data = loader.load()
data

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n      

Split the Loaded Data into Chunks

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
texts = text_splitter.split_documents(data)
texts

[Document(page_content='Fixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n                            \n\nDeposits\nCredit Card\nLoans\nInsurance\nInvestment\nRemittance\

Initialize Pinecone

In [4]:
import os
import pinecone

pinecone.init(
  api_key=os.getenv("PINECONE_API_KEY"),
  environment=os.getenv("PINECONE_ENVIRONMENT")
)
index_name = "main-index"
namespace = "hlb-fixed-deposit-with-conversation-chain-gpt-4"
if index_name not in pinecone.list_indexes():
  pinecone.create_index(
    name=index_name,
    metric="cosine",
    dimension=1536
  )
index = pinecone.Index(index_name)
index.delete(delete_all=True, namespace=namespace)

d:\Code\langchain-test\venv\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


{}

Create Embeddings and Vectors to Insert to Pinecone Database

In [5]:
from langchain_community.vectorstores.pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_store = Pinecone.from_documents(texts, embeddings, index_name=index_name, namespace=namespace)

Create LLM

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.1)

Creating Prompt Template

In [7]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_template = """Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
If there is no context, don't answer with your own knowledge, just say you don't know.
----------------
{context}"""
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]
prompt = ChatPromptTemplate.from_messages(messages)

Create Retriever for Vector Search

In [8]:
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

Compressor Pipeline

In [9]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

embedding_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.8)

Contextual Compression

In [10]:
from langchain.retrievers import ContextualCompressionRetriever

compression_retriever = ContextualCompressionRetriever(
    base_compressor=embedding_filter, base_retriever=retriever
)

Creating Memory

In [18]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(llm=llm, memory_key="chat_history", output_key="answer", k=2, return_messages=True)

Creating Chain

In [19]:
from langchain.chains import ConversationalRetrievalChain
from langchain_community.callbacks.manager import get_openai_callback
 
chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  chain_type="stuff",
  retriever=compression_retriever,
  memory=memory,
  combine_docs_chain_kwargs={"prompt": prompt},
  verbose=True,
  return_source_documents=True,
  max_tokens_limit=3000
)

def ask_question(question):
  with get_openai_callback() as cb:
    result = chain.invoke(question)
    for source_documents in result["source_documents"]:
      print(f'\n{source_documents}')
    print(f'\nDocument: {len(result["source_documents"])}')
    print(f'\nAnswer: {result["answer"]}')
    print(f'{cb}\n')

Running the Chain

In [20]:
ask_question("What is fixed deposit?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
If there is no context, don't answer with your own knowledge, just say you don't know.
----------------
Foreign Currency Fixed Deposit






More Details



A convenient way to invest in foreign currency




































Credit Card Name




















Our Artificial Intelligence Advisor is ready to assist you






Chat with us







×









×  
External Link Disclaimer


You are leaving Hong Leong Bank's website as such our Privacy Notice shall cease. We wish to remind you on our terms on the use of links, Disclaimer and Reservation of Intellectual Property Rights.



Continue
Cancel

Fixed Deposit | Hong Leong Bank Malaysia










Fixed Deposit
 
null























PERSONAL

In [21]:
ask_question("How many types of fixed deposit does HongLeong Bank provide?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is fixed deposit?
Assistant: A Fixed Deposit (FD) is a financial instrument provided by banks which offers investors a higher rate of interest than a regular savings account, until the given maturity date. It is a safer investment option when compared to stocks or bonds. With a fixed deposit, money is deposited for a specific period, which can range from a few weeks to several years, and the interest is usually paid at the end of the term (at maturity). Fixed deposits are a popular means of saving for individuals who prefer to earn a guaranteed return with low risk.
Follow Up Input: How many types of fixed deposit does HongLeong Bank provide?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain cha

In [22]:
ask_question("What is the difference between Fixed Deposit and eFixed Deposit?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is fixed deposit?
Assistant: A Fixed Deposit (FD) is a financial instrument provided by banks which offers investors a higher rate of interest than a regular savings account, until the given maturity date. It is a safer investment option when compared to stocks or bonds. With a fixed deposit, money is deposited for a specific period, which can range from a few weeks to several years, and the interest is usually paid at the end of the term (at maturity). Fixed deposits are a popular means of saving for individuals who prefer to earn a guaranteed return with low risk.
Human: How many types of fixed deposit does HongLeong Bank provide?
Assistant: Hong Leong Bank offers the following types of fixed deposit accounts:

1. Fixed Deposit - A traditional fixed depo

In [23]:
ask_question("What are the interest rates for Fixed Deposit?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How many types of fixed deposit does HongLeong Bank provide?
Assistant: Hong Leong Bank offers the following types of fixed deposit accounts:

1. Fixed Deposit - A traditional fixed deposit account with fixed tenures of up to 60 months, offering attractive interest rates.

2. e-Fixed Deposit - An online fixed deposit placement via HLB Connect, allowing customers to manage their fixed deposits online.

3. Flexi Fixed Deposit - This account provides the flexibility of making early partial withdrawals while still earning interest.

4. Senior Savers Flexi Fixed Deposit - A fixed deposit account designed for individuals aged 50 years and above, offering attractive interest rates.

5. Junior Fixed Deposit - A fixed deposit account for children below the age of 18, of

In [ ]:
ask_question("Let's say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw at the end of the term.")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the difference between Fixed Deposit and eFixed Deposit?
Assistant: Based on the provided context, there is no explicit information detailing the differences between a traditional Fixed Deposit and an e-Fixed Deposit at Hong Leong Bank. However, typically, an e-Fixed Deposit refers to a fixed deposit that can be managed online through a bank's internet banking platform, offering convenience in terms of placement, monitoring, and withdrawal without the need to visit a physical branch. Traditional Fixed Deposits might require physical visits to the bank for account management. If you need specific details about the differences at Hong Leong Bank, it would be best to contact the bank directly or visit their official website for more information.
Human: Wha

: 